Ερώτημα 2
=========

In [3]:
import pandas as pd

data = pd.read_csv("./data/onion-or-not.csv")
print("Diastaseis {}".format(data.shape))
data.head()

Diastaseis (24000, 2)


,text,label
0,Entire Facebook Staff Laughs As Man Tightens P...,1
1,Muslim Woman Denied Soda Can for Fear She Coul...,0
2,Bold Move: Hulu Has Announced That They’re Gon...,1
3,Despondent Jeff Bezos Realizes He’ll Have To W...,1
4,"For men looking for great single women, online...",1


Δημιουργία διανύσματος λέξεων

In [4]:
original_words = [text.split(" ") for text in data["text"]]

Επεξεργασία διανύσματος λέξεων

In [5]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()
processed_words = []

# Stemming
for sentence in original_words:
    processed_words.append([ps.stem(word) for word in sentence])

In [6]:
import nltk
from nltk.corpus import stopwords

# Stop words removal
try:
    stop_words = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

for i in range(len(processed_words)):
    j = 0
    while j < len(processed_words[i]):
        word = processed_words[i][j]
        if word in stop_words:
            processed_words[i].pop(j)
        j += 1

Υπολογισμός tf-idf μετρικής

In [7]:
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

# N = len(processed_words)
N = 1000
sentence_lengths = [len(processed_words[i]) for i in range(N)]
M = max(sentence_lengths)
words_num = sum(sentence_lengths)

# Custom code - Not working well
# f = np.zeros([N, M])
# for i in range(N):
#     sentence = processed_words[i]
#     for j in range(len(sentence)):
#         word = sentence[j]
#         f[i][j] = sentence.count(word) / words_num
# tf = f/f.max()


# def countApperances(i):
#     dfi = []
#     # dfi = Parallel(n_jobs=num_cores)(delayed(countWordApperances)(i,j) for j in range(len(processed_words[i])))
#     dfi = [countWordApperances(i,j) for j in range(len(processed_words[i]))]
#     # print(dfi)
#     dfi = np.array(dfi)
#     return dfi

# def countWordApperances(i,j):
#     word = processed_words[i][j]
#     dfij = np.array([word in sentence for sentence in processed_words]).sum()
#     return dfij

# num_cores = multiprocessing.cpu_count()
# # df = Parallel(n_jobs=num_cores, verbose=10)(delayed(countApperances)(i) for i in range(N))
# df = [countApperances(i) for i in range(N)]
# df = np.array(df)

# idf = np.zeros([N,M])
# idf = N/df
# idf = np.array([np.log2(idf[i]) for i in range(len(idf))])


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X = tfidf.fit_transform([" ".join(sentence) for sentence in processed_words])
Y = data["label"].to_numpy()
N = X.shape[0]
M = X.shape[1]

In [9]:
# tfidf.vocabulary_['entir']
# X[0, tfidf.vocabulary_['entir']]
print(X[0, :])
X.shape

(0, 15787)	0.29908528470290713
  (0, 13909)	0.39740171711318356
  (0, 17847)	0.49059179083294885
  (0, 10882)	0.16362201311130703
  (0, 1494)	0.2405785953388561
  (0, 10193)	0.3808735729867216
  (0, 16742)	0.34103916755488445
  (0, 6513)	0.27392624076470645
  (0, 6162)	0.3014653561878988


(24000, 19902)

## Δημιουργία μοντέλου Νευρωνικού Δικτύου

In [10]:
X = X.toarray()

In [11]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Dense(M, activation='relu', input_shape=(M,)),
    tf.keras.layers.Dropout(0.2),
    # Hidden layers
    # tf.keras.layers.Dense(M/2),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(M/4),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(M/10),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(M/20),
    tf.keras.layers.Dropout(0.2),
    # Output layer
    tf.keras.layers.Dense(2),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [13]:
print(X_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
model.fit(X_train, Y_train, epochs=5)

Train on 18000 samples
Epoch 1/5
18000/18000 [==============================] - 750s 42ms/sample - loss: 0.6960 - accuracy: 0.6244
Epoch 2/5
18000/18000 [==============================] - 748s 42ms/sample - loss: 0.6930 - accuracy: 0.6249
Epoch 3/5
18000/18000 [==============================] - 691s 38ms/sample - loss: 0.6930 - accuracy: 0.6249
Epoch 4/5
18000/18000 [==============================] - 669s 37ms/sample - loss: 0.6930 - accuracy: 0.6249
Epoch 5/5
18000/18000 [==============================] - 666s 37ms/sample - loss: 0.6930 - accuracy: 0.6249


In [ ]:
# Save model for later user to avoid long training time
model.save('./models/onion-or-not-model.h5')

In [16]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix

Y_predicted = model.predict(X_test)
print(classification_report(Y_test, Y_predicted))

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets